In [1]:
from my_models import *
from sklearn.manifold import TSNE
import numpy as np
from  matplotlib import pyplot as plt
from Config import Config
from tensorflow.keras.utils import to_categorical

In [2]:
DATA_ROOT = ''
DATA_PATH = ''
CLASS_LABELS = Config.RAVDESS_LABELS
k = 10

h5_path = ''
feature_name = 'mfcc'

learning_rate=Config.LR
beta_1=Config.BETA_1
beta_2=Config.BETA_2
epsilon=Config.EPS

EPOCHS = Config.EPOCHS
BATCH = Config.BATCH

In [3]:
# Read data
with open(f'{DATA_ROOT}/{DATA_PATH}.npy', 'rb') as f:
    x = np.load(f)
    y = np.load(f)

y = to_categorical(y,num_classes=len(CLASS_LABELS))

In [4]:
from sklearn.model_selection import KFold
import tensorflow as tf

x_test = None
y_test = None

kfold = KFold(n_splits=k, shuffle=True, random_state=98)

for (train, test) in kfold.split(x, y):
    x_test, y_test = x[test], y[test]
    x_test = tf.expand_dims(x_test, axis=-1)
    break

In [5]:
origin = Model(len(CLASS_LABELS))
origin.build(input_shape=x_test.shape)
origin.load_weights(h5_path)

In [6]:
def get_layers_weights(origin_model, until):
    """get weights of model

    Args:
        origin_model (keras.Model): Original model (+weights)
        until (int): Index of wanted final layer

    Returns:
        list: weights for each layer
    """
    
    ret = []
    
    copy_layers = origin_model.layers[:until]
    for cur_layer in copy_layers:
        ret.append(cur_layer.get_weights())
    
    return ret

In [7]:
origin_layers_weights = get_layers_weights(origin, until=-3)

In [9]:
test_model = Model2(len(CLASS_LABELS))
test_model.build(input_shape=x_test.shape)

In [10]:
test_model_layers = test_model.layers
for i, cur_test_layer in enumerate(test_model_layers):
    cur_test_layer.set_weights(origin_layers_weights[i])

In [11]:
predictions = test_model(x_test, training=False)
print(predictions)

tf.Tensor(
[[[-0.14912269 -0.369866    0.09946968 ... -0.0141987  -0.03412911
   -0.5156893 ]
  [-0.18551564  0.11585237  0.13074192 ... -0.06841514  0.12432949
   -0.20623758]
  [-0.16481635 -0.10430894  0.08677782 ... -0.2754196   0.0150083
   -0.49919906]
  [-0.20427063 -0.20570779  0.17916486 ...  0.02733971 -0.17462689
   -0.26186895]
  [-0.29910025  0.23996413 -0.00325908 ... -0.19769093 -0.0368591
    0.23208445]
  [-0.11909958  0.21845171 -0.06637171 ... -0.1699413  -0.00907672
   -0.07459307]]

 [[ 0.21511136 -0.03837761  0.02199976 ...  0.27658638  0.00354105
    0.35775375]
  [ 0.09731393  0.34133822  0.05633474 ... -0.22822872  0.11266917
   -0.0131054 ]
  [ 0.1491434   0.4467044  -0.1292635  ...  0.1368583   0.19754949
    0.05897012]
  [ 0.15638027 -0.08815068 -0.34406325 ...  0.13796254 -0.20561704
    0.13539708]
  [-0.18490602  0.2671858   0.01129091 ... -0.19687413 -0.06091921
    0.27848244]
  [ 0.03618377  0.19155489 -0.03869793 ... -0.1991206  -0.06489649
   -0.038

In [12]:
len(predictions)

144